In [ ]:
from mysql.mysql  import Mysql
import minio 
import time
import sys
from toollib.guid  import SnowFlake
import traceback
import hashlib
curPath = '/Users/hcbigboss/RESP/html/raspberry/script'

if curPath not in sys.path:
    sys.path.append(curPath)

import FileUtil

In [ ]:
mysqlHandle = Mysql('../configs/mysql.yml')

cursor = mysqlHandle.getCursor('filedb')


minioHandle = minio.Minio('192.168.0.151:9000','adminminio','adminminio',secure=False)


snowFlake = SnowFlake()

In [ ]:
def dFFiles(bucket,filePath,dirList,fileList,parentId):

    files =  minioHandle.list_objects(bucket,filePath)
    
    for file in files:
        id = snowFlake.gen_uid()
        fileName =  file.object_name
        rel_path = '/'+ bucket+'/'+ file.object_name
        create_time = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())
        modify_time = create_time
        type = ''

        if file.is_dir:
            fileName = fileName[0:len(fileName)-1]
            pos =fileName.rfind('/')
            title = fileName if pos == -1 else fileName[pos+1:]
            dirList.append((
                id,
                title,
                rel_path,
                parentId,
                create_time,
                modify_time,
                type
            ))
            dFFiles(bucket,file.object_name,dirList,fileList,id)
        else:
            pos =fileName.rfind('/')
            title = fileName[pos+1:]
            pos = title.rfind('.')
            type = title[pos:]
            fileList.append((
                id,
                title,
                rel_path,
                parentId,
                create_time,
                modify_time,
                type
            ))

    return dirList,fileList

def getFiles(bucket,filePath,parentId):
    fileList = []
    dirList = []
    dFFiles(bucket,filePath,dirList,fileList,parentId)
    return dirList,fileList



In [34]:
def writeToDb( dbName,dirList,fileList):
    try:
        sql = "insert into dirs (`id`,`title`,`rel_path`,parent_id,create_time,modify_time,type) values(%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(sql,dirList)

        sql = "insert into " + dbName + " (`id`,`title`,`rel_path`,parent_id,create_time,modify_time,type) values(%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(sql,fileList)
        # cursor.executemany(sql,fileList)
        mysqlHandle.commit('filedb')
    except Exception as e:
        mysqlHandle.rollback('filedb')
        traceback.print_exc()
        
        
    pass

In [ ]:

dirList,fileList = getFiles('video','/',None)
print(dirList)
print("====================")
writeToDb('videos',dirList,fileList)

# for dir in dirList:
#     print(dir.object_name)

# print('-------------')
# # print(fileList)
# for file in fileList:
#     print(file.object_name)


In [35]:
dirList,fileList = getFiles('picture','/',None)
print(dirList)
print("====================")
writeToDb('images',dirList,fileList)

2023-02-26 22:28:09,980 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /picture?location= HTTP/1.1" 200 128
2023-02-26 22:28:09,994 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /picture?delimiter=%2F&encoding-type=url&list-type=2&max-keys=1000&prefix=%2F HTTP/1.1" 200 351
2023-02-26 22:28:10,114 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /picture?delimiter=%2F&encoding-type=url&list-type=2&max-keys=1000&prefix=saigao%2F HTTP/1.1" 200 7072
2023-02-26 22:28:10,137 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /picture?delimiter=%2F&encoding-type=url&list-type=2&max-keys=1000&prefix=saigao%2F1466%20%E8%B5%9B%E9%AB%98%E9%85%B1-NO.059%202019%E5%9C%A3%E8%AF%9E%E7%9B%B4%E6%92%AD%EF%BC%9A%E8%BD%BBSM%E5%85%BD%E8%80%B3%E5%B0%BE%E5%B7%B4%20%E7%B4%AB%E8%89%B2%E8%A1%AC%E8%A1%A3%20%5B99P1V-152MB%5D%2F HTTP/1.1" 200 53653
2023-02-26 22:28:10,164 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /picture

[(159952509316628480, 'saigao', '/picture/saigao/', None, '2023-02-26 22:28:09', '2023-02-26 22:28:09', ''), (159952509819944960, '1466 赛高酱-NO.059 2019圣诞直播：轻SM兽耳尾巴 紫色衬衣 [99P1V-152MB]', '/picture/saigao/1466 赛高酱-NO.059 2019圣诞直播：轻SM兽耳尾巴 紫色衬衣 [99P1V-152MB]/', 159952509316628480, '2023-02-26 22:28:10', '2023-02-26 22:28:10', ''), (159952509962551396, '19邻家女孩粉白死库水 52图 4花絮视屏', '/picture/saigao/19邻家女孩粉白死库水 52图 4花絮视屏/', 159952509316628480, '2023-02-26 22:28:10', '2023-02-26 22:28:10', ''), (159952510042243113, '2018单身节特刊-水龙敬巫女性感内衣(128P 5V)', '/picture/saigao/2018单身节特刊-水龙敬巫女性感内衣(128P 5V)/', 159952509316628480, '2023-02-26 22:28:10', '2023-02-26 22:28:10', ''), (159952510151295046, '2019七夕特典： 黑长直色气loli水手服写真 200图  + 6长视频', '/picture/saigao/2019七夕特典： 黑长直色气loli水手服写真 200图  + 6长视频/', 159952509316628480, '2023-02-26 22:28:10', '2023-02-26 22:28:10', ''), (159952510411341992, '2019端午节特刊两套', '/picture/saigao/2019端午节特刊两套/', 159952509316628480, '2023-02-26 22:28:10', '2023-02-26 22:28:10', ''), (159952510

In [33]:
files =  minioHandle.list_objects('video','/',True)

for file in files:
    # pos = file.object_name.rindex('/')
    # print(file.object_name[pos+1:])
    print(file.object_name)
    # if not file.is_dir :
    #  obj =minioHandle.get_object('video',file.object_name)
    #  hash =  hashlib.md5()
    #  hash.update(obj.data)
    #  print(hash.hexdigest())
    

2023-02-26 22:21:58,682 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (82): 192.168.0.151:9000
2023-02-26 22:21:58,737 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video?delimiter=&encoding-type=url&list-type=2&max-keys=1000&prefix=%2F HTTP/1.1" 200 42850
2023-02-26 22:21:58,764 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/29%20%E5%A4%A7%E9%95%BF%E8%85%BF%E5%B0%8F%E5%A7%90%E5%A7%90%20%E8%AF%B1%E4%BA%BA%E7%9A%84%E9%B2%8D%E9%B1%BC%20%E5%A4%A7%E5%B1%8C%E4%BD%BF%E5%8A%B2%E5%B9%B2%E5%A5%B9%E7%9A%84%E5%B0%8F%E9%AA%9A%E9%80%BC%20%E6%93%8D%E5%AE%8C%E7%94%A8%E9%87%91%E6%89%8B%E6%8C%87%E7%BB%A7%E7%BB%AD%E7%88%BD/.120917.torrent HTTP/1.1" 200 39374
2023-02-26 22:21:58,779 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/29%20%E5%A4%A7%E9%95%BF%E8%85%BF%E5%B0%8F%E5%A7%90%E5%A7%90%20%E8%AF%B1%E4%BA%BA%E7%9A%84%E9%B2%8D%E9%B1%BC%20%E5%A4%A7%E5%B1%8C%E4%BD%BF%E5%8A%B2%E5%B9%B2%E5%A5%B9%E7%9A%84%E5%B0%8F%E9%AA%

82e4ab58cfe05974130a00a674a765c1
95caaa46317b5827808cb2491803ec25


2023-02-26 22:22:00,845 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/aki%E7%A7%8B%E6%B0%B4%20%E5%8F%8C%E9%AB%98%E8%B7%9F%E9%9E%8B%E7%9A%84%E9%9E%8B%E9%9D%A2%E6%95%B2%E5%87%BB%E4%BB%A5%E5%8F%8A%E8%AF%95%E7%A9%BF%E8%B5%B0%E8%B7%AF%E5%A3%B0%E9%9F%B3%20%E9%BB%91%E4%B8%9D%20%E7%99%BD%E4%B8%9D%20jiojio%E7%88%B1%E5%A5%BD%E8%80%85%E5%85%A5%EF%BC%81%E3%80%901asmr.top%E3%80%91/115%E8%B5%84%E6%BA%90%E8%BD%AC%E5%AD%98%E6%96%B9%E6%B3%95.txt HTTP/1.1" 200 35
2023-02-26 22:22:00,860 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/aki%E7%A7%8B%E6%B0%B4%20%E5%8F%8C%E9%AB%98%E8%B7%9F%E9%9E%8B%E7%9A%84%E9%9E%8B%E9%9D%A2%E6%95%B2%E5%87%BB%E4%BB%A5%E5%8F%8A%E8%AF%95%E7%A9%BF%E8%B5%B0%E8%B7%AF%E5%A3%B0%E9%9F%B3%20%E9%BB%91%E4%B8%9D%20%E7%99%BD%E4%B8%9D%20jiojio%E7%88%B1%E5%A5%BD%E8%80%85%E5%85%A5%EF%BC%81%E3%80%901asmr.top%E3%80%91/aki%E7%A7%8B%E6%B0%B4%20%E5%8F%8C%E9%AB%98%E8%B7%9F%E9%9E%8B%E7%9A%84%E9%9E%8B%E9%9D%A2%E6%95%B2%E5%87%BB%E4%BB%A5%E5%8F%8A%E

3e8dc26bdd665cf8dd2cb8b9933fdd50
c0807ee632b030f74fcd5c94e76e4fe8


2023-02-26 22:22:04,119 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/aki%E7%A7%8B%E6%B0%B4%20%E5%8F%8C%E9%AB%98%E8%B7%9F%E9%9E%8B%E7%9A%84%E9%9E%8B%E9%9D%A2%E6%95%B2%E5%87%BB%E4%BB%A5%E5%8F%8A%E8%AF%95%E7%A9%BF%E8%B5%B0%E8%B7%AF%E5%A3%B0%E9%9F%B3%20%E9%BB%91%E4%B8%9D%20%E7%99%BD%E4%B8%9D%20jiojio%E7%88%B1%E5%A5%BD%E8%80%85%E5%85%A5%EF%BC%81%E3%80%901asmr.top%E3%80%91/%E5%85%8D%E8%B4%B9ASMR%E8%B5%84%E6%96%99%EF%BC%8C%E5%B0%BD%E5%9C%A81asmr.top.txt HTTP/1.1" 200 77
2023-02-26 22:22:04,145 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/aki%E7%A7%8B%E6%B0%B4%20%E5%8F%8C%E9%AB%98%E8%B7%9F%E9%9E%8B%E7%9A%84%E9%9E%8B%E9%9D%A2%E6%95%B2%E5%87%BB%E4%BB%A5%E5%8F%8A%E8%AF%95%E7%A9%BF%E8%B5%B0%E8%B7%AF%E5%A3%B0%E9%9F%B3%20%E9%BB%91%E4%B8%9D%20%E7%99%BD%E4%B8%9D%20jiojio%E7%88%B1%E5%A5%BD%E8%80%85%E5%85%A5%EF%BC%81%E3%80%901asmr.top%E3%80%91/%E5%BF%85%E8%AF%BB.txt HTTP/1.1" 200 75
2023-02-26 22:22:04,158 - urllib3.connectionpool - DEBUG - http://

...

3da63875c5807f6e6040eab33ae80e75
953fb0c6ed7cd2ac23e3206f418ffdb4
4dd4a6c2c84a7468ee7aca0c42393a3e


2023-02-26 22:22:12,500 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/%E8%99%8E%E7%89%99%E9%98%BF%E7%A8%80%E7%A8%80%E5%A4%A7%E9%AD%94%E7%8E%8B%20%E8%95%BE%E4%B8%9D%E6%89%8B%E5%A5%97%2B%E8%A7%86%E9%A2%91%E8%B6%85%E8%BF%91%E8%B8%A9%E8%B8%8F%E8%A7%86%E8%A7%92%EF%BC%81%2020221027%E3%80%901asmr.top%E3%80%91/%E8%99%8E%E7%89%99%E9%98%BF%E7%A8%80%E7%A8%80%E5%A4%A7%E9%AD%94%E7%8E%8B%20%E8%95%BE%E4%B8%9D%E6%89%8B%E5%A5%97%20%E8%A7%86%E9%A2%91%E8%B6%85%E8%BF%91%E8%B8%A9%E8%B8%8F%E8%A7%86%E8%A7%92%EF%BC%81%20%281%29.mp4 HTTP/1.1" 200 83210304


e6afe12568a1f1b8d60a2ed099a04ea6


2023-02-26 22:22:14,065 - urllib3.connectionpool - DEBUG - http://192.168.0.151:9000 "GET /video/%E8%99%8E%E7%89%99%E9%98%BF%E7%A8%80%E7%A8%80%E5%A4%A7%E9%AD%94%E7%8E%8B%20%E8%95%BE%E4%B8%9D%E6%89%8B%E5%A5%97%2B%E8%A7%86%E9%A2%91%E8%B6%85%E8%BF%91%E8%B8%A9%E8%B8%8F%E8%A7%86%E8%A7%92%EF%BC%81%2020221027%E3%80%901asmr.top%E3%80%91/%E8%99%8E%E7%89%99%E9%98%BF%E7%A8%80%E7%A8%80%E5%A4%A7%E9%AD%94%E7%8E%8B%20%E8%95%BE%E4%B8%9D%E6%89%8B%E5%A5%97%20%E8%A7%86%E9%A2%91%E8%B6%85%E8%BF%91%E8%B8%A9%E8%B8%8F%E8%A7%86%E8%A7%92%EF%BC%81.mp4 HTTP/1.1" 200 16005004


217ccf2d0af56b579fb427518542da5b
d1f6688c6c744c02858ad4e7b583e23c


In [ ]:

print(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))

In [ ]:

msg = 'aaa.mp4'

# pos = msg.rindex('.')

# print(msg[pos:])
print(msg[0:len(msg)-1])